In [1]:
%matplotlib qt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py
import os
from scipy.stats import binned_statistic
from scipy.optimize import curve_fit
import concurrent
import pickle

import matplotlib
font = {'size'   : 16}
matplotlib.rc('font', **font)

### Get the file path

In [2]:
pole = 2
#file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\rl_response_data_pole{}.h5".format(pole)
file_path = r"C:\Users\3DStation4\PycharmProjects\pythonProject_3DMOKE_new\rl_response_data_pole{}.h5".format(pole)
print(os.path.isfile(file_path))
out_folder = 'results'
try:
    os.stat(out_folder)
except:
    os.mkdir(out_folder) 
poles = [0, 1, 2]
poles_dict = {0:'A', 1:'B', 2:'C'}

True


### Get the real resistance

In [3]:
from scipy.optimize import curve_fit

plt.close('all')
fig = plt.figure(figsize=(20, 5))
R_poles = np.zeros(3)
for pole in poles:
    #file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\rl_response_data_pole{}.h5".format(pole)
    file_path = r"C:\Users\3DStation4\PycharmProjects\pythonProject_3DMOKE_new\rl_response_data_pole{}.h5".format(pole)
    with h5py.File(file_path, 'r') as f:
        const_keys = list(f['dc_response'].keys())
        I = np.zeros(len(const_keys))
        V = np.zeros(len(const_keys))
        for i, c in enumerate(const_keys):
            hx = f['dc_response/'+c+'/hexapole/data'][:]
            v = f['dc_response/'+c+'/v_measurement/data'][:]
            I[i] = np.mean(hx[:, pole+1])
            #V[i] = -np.mean(v[:, 1])   # used by Luka
            V[i] = np.mean(v[:, 1])
    
            
    plt.subplot(1, len(poles), pole+1)
    plt.scatter(V, I, label='Data')   
    plt.xlabel('Voltage [V]')
    plt.ylabel('Current [A]')
    
    
    f = lambda x, a: a*x
    R, Rerr = curve_fit(f, I, V)
    R = R[0]
    Rerr = np.sqrt(np.diag(Rerr))[0]
    print('Resistance R:{:.4f}'.format(R))
    R_poles[pole] = R
    
    print('Error Rerr:{:}'.format(Rerr))
    x = np.linspace(0, np.max(V), 100)
    plt.plot(x, x/R, color='r', label='Ohms law fit')
    
    plt.legend()
    plt.title('Resistance {}: {:.3f} \u00B1 {:.3f} \u03A9'.format(poles_dict[pole], R, Rerr))

plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'RL response pole.png'))


Resistance R:2.2187
Error Rerr:0.0012026408301546606
Resistance R:2.2101
Error Rerr:0.0011503289699167616
Resistance R:2.3327
Error Rerr:0.0012014594443792476


### Getting the frequency response

In [4]:
def extract_max_amp(data, timestep=1/10000, cutoff_freq=50):
    """Extracts the frequency from the loops experiment given the instrument group.
        Returns an array frequency, max amp, phase
    """
    
    # extract fft
    n = data.shape[0]
    fft = np.fft.rfft(data, axis=0)
    freq = np.fft.rfftfreq(n, d=timestep)
    fft_amp = np.absolute(fft)
    
    fft[freq>cutoff_freq] = 0
    data_clean = np.fft.irfft(fft, axis=0, n=n)
    
    # get the maximum amplitude
    indx = np.unravel_index(np.argmax(fft_amp, axis=None), fft_amp.shape)
    return freq[indx[0]], (np.max(data_clean) - np.min(data_clean))/2

def extract_loop_data(grp, instrument):
    data = np.vstack([grp[key][instrument]['data'][:] for key in grp.keys()])
    return data


def get_frequency_data(file_path, pole='0'):
    """Gets the fourier data from the frequency response experiment for the given pole of the magnet given the file path"""
    instruments=['v_measurement', 'hexapole']
    data = {key:[] for key in instruments}
    with h5py.File(file_path, 'r') as f:
        grp_loops = f['frequency_response/pole'+pole]
        n_loops = len(grp_loops.keys())
        for i, loops in enumerate(grp_loops.keys()):
            for inst in instruments:
                # get the data for the instrument
                loop_data = extract_loop_data(grp_loops[loops], inst)
                freq, max_amp = extract_max_amp(loop_data[:, 1:])
                period = grp_loops[loops].attrs['period']
#                 freq = 1/period
                data[inst].append(np.array([freq, max_amp]))
    for inst in data:
        data[inst] = np.vstack(data[inst])
    
    return data

def format_freq_data(freq_data):
    """Formats the data and gets rid of the non matching data"""
    data_out = pd.DataFrame(np.hstack((freq_data['hexapole'], freq_data['v_measurement'])), 
                            columns=['i_freq', 'i_amp', 'v_freq', 'v_amp'])
    # make sure that all of the data matches (i.e. hallprobe and hexapole frequencies corresponse)
    freq_delta = np.abs(data_out['i_freq'] - data_out['v_freq'])
    print('There are {} data points where frequencies do not match.'.format(np.sum(freq_delta>=1/100)))
    data_out = data_out.loc[freq_delta<1/100, :]
    # drop one of the frequencies and calculate the phase delta
    data_out.drop(columns='v_freq', inplace=True)
    data_out.rename(columns={'i_freq':'freq'}, inplace=True)
    return data_out

def get_formatted_data(file_path, pole):
    data = get_frequency_data(file_path, pole=str(pole))
    return format_freq_data(data)

### Check the data extraction

In [5]:
pole = 0
loops = 10
#file_path = r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\rl_response_data_pole{}.h5".format(pole)
file_path = r"C:\Users\3DStation4\PycharmProjects\pythonProject_3DMOKE_new\rl_response_data_pole{}.h5".format(pole)

with h5py.File(file_path, 'r') as f:
    instruments=['v_measurement', 'hexapole']
    data = {key:[] for key in instruments}
    loop_data = {key:[] for key in instruments}
    grp_loops = f['frequency_response/pole'+str(pole)]
    n_loops = len(grp_loops.keys())
    grp_loops = list(grp_loops.values())
#     for i, loops in enumerate(grp_loops.keys()):
    for inst in instruments:
        # get the data for the instrument
        loop_data[inst] = extract_loop_data(grp_loops[loops], inst)
        freq, max_amp = extract_max_amp(loop_data[inst][:, 1:])
        period = grp_loops[loops].attrs['period']
#                 freq = 1/period
        data[inst].append(np.array([freq, max_amp]))
    for inst in data:
        data[inst] = np.vstack(data[inst])
    

In [6]:
data

{'v_measurement': array([[0.47499846, 6.13466096]]),
 'hexapole': array([[0.47499846, 2.67777903]])}

In [7]:
plt.close('all')
plt.figure()
for inst in instruments:
    t = loop_data[inst][:, 0]
    t -= t[0]
    plt.plot(t, loop_data[inst][:, pole+1], label=inst)
    plt.scatter(np.hstack([t, t]), data[inst][0, 1]*np.hstack([1+0*t, -1+0*t]), label=inst+'_amplitude')
    
plt.legend(loc='upper left')
plt.xlabel('t [s]')
plt.ylabel('signal [V]')
plt.tight_layout()

In [8]:
data

{'v_measurement': array([[0.47499846, 6.13466096]]),
 'hexapole': array([[0.47499846, 2.67777903]])}

In [9]:
vdata = loop_data['hexapole'][:, 1:]
timestep = t[1] - t[0]
# extract fft
n = vdata.shape[0]
fft = np.fft.rfft(vdata, axis=0) / (n / 2)
fft_phase = np.angle(fft)
fft_amp = np.absolute(fft)
freq = np.fft.rfftfreq(n, d=timestep)

# get the maximum amplitude
indx = np.unravel_index(np.argmax(fft_amp, axis=None), fft_amp.shape)
print(freq[indx[0]])

fft[freq>10, :] = 0
vdata_clean = np.fft.irfft(fft, n=n, axis=0) * (n / 2)

# get the maximum amplitude
indx = np.unravel_index(np.argmax(fft_amp, axis=None), fft_amp.shape)
print(np.array([freq[indx[0]], fft_amp[indx], fft_phase[indx]]))

0.47499845637425153
[ 0.47499846  2.67777334 -1.57145761]


In [10]:
plt.plot(t, vdata)
plt.plot(t, vdata_clean[:])

### Run the extraction using the fft

In [11]:
frequency = 0.1 + 3 * np.linspace(0, 1, 9)
frequency

array([0.1  , 0.475, 0.85 , 1.225, 1.6  , 1.975, 2.35 , 2.725, 3.1  ])

In [12]:
freq_data = [None]*3
def get_file_path(pole):
    #return r"C:\Users\lukas\OneDrive - University of Cambridge\PhD\3DMOKE\Data\MagnetCalibration\rl_response_data_pole{}.h5".format(pole)
    return r"C:\Users\3DStation4\PycharmProjects\pythonProject_3DMOKE_new\rl_response_data_pole{}.h5".format(pole)


with concurrent.futures.ThreadPoolExecutor() as executor:
    get_data_proxy = lambda p: get_formatted_data(get_file_path(p), p)
    for pole, result in zip(poles, executor.map(get_data_proxy, poles)):
        freq_data[pole] = result

print('Done!')
# freq_data = get_formatted_data(file_path, pole)

There are 0 data points where frequencies do not match.
There are 0 data points where frequencies do not match.
There are 0 data points where frequencies do not match.
Done!


### Save the result

In [13]:
# for pole in poles:
#     freq_data[pole]['phase_delta']-=2*np.pi
with open(os.path.join(out_folder,'frequency_data3.p'.format(pole)), 'wb') as fp:
    pickle.dump(freq_data, fp, protocol=4)
print('Saved')

Saved


### Load the result if have saved

In [14]:
with open(os.path.join(out_folder,'frequency_data3.p'.format(pole)), 'rb') as fp:
    freq_data = pickle.load(fp)

### Plot the response

In [15]:
for pole in poles:
    plt.scatter(freq_data[pole]['v_amp'], freq_data[pole]['i_amp'], c=freq_data[pole]['freq'])
    plt.xlabel('Voltage [V]')
    plt.ylabel('Current [A]')
    plt.title('Pole {} Amplitude Response'.format(poles_dict[pole]))
    plt.tight_layout()
    plt.savefig(os.path.join(out_folder, 'Amplitude_response_pole{}.png'.format(poles_dict[pole])))

### Fit the amplitudes

In [16]:
R_poles

array([2.21873049, 2.21005188, 2.33272432])

In [17]:
plt.close('all')
fig = plt.figure(figsize=(20, 5))
L_poles = np.zeros(3)
for pole in poles:
    with open(os.path.join(out_folder,'frequency_data3.p'.format(pole)), 'rb') as fp:
        freq_data = pickle.load(fp)
    
    R = R_poles[pole]

    print('Pole {}'.format(poles_dict[pole]))
    V = freq_data[pole]['v_amp'].values
    f = freq_data[pole]['freq'].values
    I = freq_data[pole]['i_amp'].values

    # filter out the data where the load is too big. Here assuming a large inductance of 1H
    indx = np.logical_and(I*np.sqrt(R**2+(np.pi*2*f)**2) < 15, V > 0)
    f = f[indx]
    V = V[indx]
    I = I[indx]

    x =np.hstack((f[:, np.newaxis], I[:, np.newaxis]))

    amp_fitting = lambda x, L: x[:, 1]*np.sqrt(R**2+(np.pi*2*L*x[:, 0])**2)
    popt, perr = curve_fit(amp_fitting, x, V)#, p0 = [0.01, 0.01], bounds=[[0, 0], [100, 100]])#, max_nfev=10000, diff_step=0.001)
    amp_fun = lambda x: amp_fitting(x, *popt)
    L = popt[0]
    Lerr = np.sqrt(np.diag(perr))[0]
    print('R:{}'.format(R))
    print('L:{}'.format(L))
    print('L error:{}'.format(perr[0][0]))
    L_poles[pole] = L
    
    error = amp_fun(x) - V
    rel_error = np.abs(error)/V
    print('Fit std: ', np.std(error))

    plt.subplot(1, len(poles), pole+1)
    plt.scatter(f, amp_fun(x), c='red', alpha=0.7, label='Fit, L={:.4f} H'.format(L))
    plt.scatter(f, V, c=I, marker='+', s=100, label='Data')

    plt.xlabel('Frequency [Hz]')
    plt.ylabel('Voltage amplitude [V]')
    plt.title('Pole {} Amplitude Response Fit.\nInductance: {:.3f} \u00B1 {:.3f} H'.format(poles_dict[pole], L, Lerr))
    plt.legend(loc='upper right')
plt.tight_layout()
plt.savefig(os.path.join(out_folder, 'Amplitude_response_freq_fit_improved.png'))
    # plt.savefig(os.path.join(out_folder, 'Small-amplitude_response_pole{}_fit.png'.format(poles_dict[pole])))

Pole A
R:2.2187304880211283
L:0.15845467489203932
L error:2.7759851300567324e-06
Fit std:  0.04324086464697541
Pole B
R:2.2100518797834403
L:0.18295699563985293
L error:4.689736156527595e-06
Fit std:  0.06332928559788502
Pole C
R:2.3327243244900413
L:0.1759026185972536
L error:3.7274110150871672e-06
Fit std:  0.05237413752900019


In [18]:
err_tbl = pd.DataFrame({'error':error, 'rel_error':rel_error, 'V':V, 'I':I, 'freq':f})
err_tbl.sort_values('rel_error', ascending=False)

,error,rel_error,V,I,freq
48,0.053845,0.149306,0.360633,0.100000,3.099814
44,0.041677,0.122842,0.339270,0.100001,2.724919
40,0.031160,0.098004,0.317948,0.100001,2.349992
35,0.021513,0.072200,0.297963,0.100000,1.975016
49,0.100104,0.060672,1.649916,0.422224,3.099814
29,0.012705,0.045371,0.280019,0.100000,1.600000
45,0.068404,0.044417,1.540036,0.422227,2.724919
50,0.110899,0.037281,2.974662,0.744447,3.099814
41,0.041641,0.029071,1.432370,0.422225,2.349992
46,0.066424,0.023984,2.769510,0.744453,2.724919


### Save the final RL coefficients

In [19]:
# rl_coeff = [
#     {'R':2.124, 'L':0.2021},  
#     {'R':2.115, 'L':0.2047},
#     {'R':2.082, 'L':0.1966}
# ]
rl_coeff = [{'R': R, 'L': L} for R, L in zip(R_poles, L_poles)]
with open('rl_coeff.p', 'wb') as f:
    pickle.dump(rl_coeff, f)

In [20]:
rl_coeff

[{'R': 2.2187304880211283, 'L': 0.15845467489203932},
 {'R': 2.2100518797834403, 'L': 0.18295699563985293},
 {'R': 2.3327243244900413, 'L': 0.1759026185972536}]